<a href="https://colab.research.google.com/github/anilmm2005/stock/blob/master/automate_fin_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Step 1: Obtain list of ticker symbols for securities of interest
The first step is to gather the list of Ticker Symbols (the stock identifier for each security) for the relevant list of companies; in this case, I am using the SP500 Index components, whose list is readily available online at the relevant Wikipedia page.

1. Import relevant Packages: As a first step , you will need to import the relevant Python packages and define the url variable, which points to the online data source. You should also import the pandas package (“import pandas as pd”) as you will need it later on.


In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

2. Scrape data: You then need to fetch the relevant url content with the request.get method. Then, you can extract its text with the .text method, and finally parse it into a Beatiful Soup object.

In [0]:
r = requests.get(url,timeout=2.5)
r_html = r.text
soup = BeautifulSoup(r_html, 'html.parser')
components_table = soup.find_all(id="constituents")
headers_html = soup.find_all("th")
df_columns=[item.text.rstrip("\n") for item in headers_html]

3. Store data into a Pandas Dataframe: next, you need to isolate the first table by indexing the components_table object (which resides at index 0), and then find all the rows within the body of the table, which are not headers; go ahead and store the results into data_rows.

In [23]:
components_headers=df_columns[:9]
data_rows=components_table[0].find("tbody").find_all("tr")[1:]
rows=[]
for row in range(len(data_rows)):
    
    stock=list(filter(None,data_rows[row].text.split("\n")))
    rows.append(stock)
S_P_500_stocks=pd.DataFrame(rows,columns=components_headers)
S_P_500_stocks

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,None
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969
502,ZBH,Zimmer Biomet Holdings,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,None
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,0000109380,None


4. Save dataset into csv file: as shown in the code below, you will then remove the redundant column named SEC Filings, and use the .to_csv method to save the dataset locally. You can now use this table to fetch stock price data from the API.

In [24]:
S_P_500_stocks.drop("SEC filings",inplace=True,axis=1)
S_P_500_stocks.to_csv(r"/content/drive/My Drive/Colab Notebooks/SP500stocks.csv",index=False)
S_P_500_stocks

,Symbol,Security,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989
...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,None
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969
502,ZBH,Zimmer Biomet Holdings,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,None
503,ZION,Zions Bancorp,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,0000109380,None


In [6]:
S_P_500_stocks

,Symbol,Security,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989
...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,None
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969
502,ZBH,Zimmer Biomet Holdings,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,None
503,ZION,Zions Bancorp,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,0000109380,None


Step 2: Collect historical stock price data using the Tiingo API
Tiingo is a financial markets API which provides a variety of information relating to publicly-listed companies over several markets; in this example, we are interested in collecting the daily historical stock price information (Date, Open, Close, High, Low, Volume) and will therefore refer to the related Tiingo API documentation.

1.Import relevant Packages: As before, proceed to import the necessary packages. This time, you’ll additionally need to import the datetime module, as you will use later on to define the time window along which we want to extract data from the API.

In [0]:
import requests
import pandas as pd
import datetime
from datetime import datetime,timedelta


In [0]:
#Tiingo API
token1="fa74d4a6666bf67e91c5167983ff54e4d24f6285"


3. Import the SP500 members table yousaved as a csv file in Step 1; this can be easily done with the .read_csv method in pandas.

In [0]:
SP=pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/SP500stocks.csv")
SP

4. Call the Tiingo API and get historical stock price data You first have to set up a list of the ticker symbols to iterate over in order to make an API call for each of them, storing them in a variable ticker_symbols, which you can extend with the addition of the SP500 Index value (“SPY”) in the following line.


In [13]:
ticker_symbols=list(SP["Symbol"])
print(ticker_symbols)
ticker_symbols.append("SPY")
print(ticker_symbols)
end_date=str(datetime.now().date()-timedelta(days=1))
end_date

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA

'2020-06-10'

The next lines of code are the ones where you will actually call the Tiingo API to get data for each ticker symbol of interest. First, you need to set the symbol, end_date and token parameter in the Tiingo url to submit into the request.get method; you then need to store the response into a variable r, which corresponds to the csv file containing the stock price history for the given ticker. The following few lines in the code below convert such values into a list then a dataframe df, which you will then append to the data1 collection before moving on to the next ticker_symbol in iterative fashion.


In [31]:
data1=[]
count=0
for symbol in ticker_symbols:
    try:
        url = "https://api.tiingo.com/tiingo/daily/{}/prices?startDate=2000-01-03&endDate={}&format=csv&token={}".format(symbol,end_date,token1)
        url
        r = requests.get(url,timeout=10)
        #print(r)
        rows=r.text.split("\n")[1:-1]
        df=pd.DataFrame(rows)
        df=df.iloc[:,0].str.split(",",13,expand=True)
        df["Symbol"]=symbol
        data1.append(df)
        count+=1
        if count == 5:
            break

SyntaxError: ignored

In [12]:
print(data1)

[]
